# CMPT 423/820: Assignment 3  Question 6 
* Your name
* Your NSID

In this Notebook, you'll use Python to explore Structure Learning in Bayesian Networks.  The basic approach will be:
1. Load a categorical dataset (`diamond.csv`) 
3. Create 2 (related) Bayesian network models, fitting their parameters to the data.
4. Compare the 2 structures, using log-likelihood.

The data has 5 columns, with the first column being an index.  The first row gives the column names, $A, B, C, D$.  There are too many rows and columns to calculated this data by hand, so we will need to use computational tools.

In [8]:
import pandas as pd

dataframe = pd.read_csv('diamond.csv', index_col=0)

## Two Bayesian network structures
We will look at two Bayesian network structures out of the very many that are possible for this dataset.
2. Model $M_1$.
$$ P_1(ABCD) = P(A)P(B|A)P(C|A)P(D|BC) $$
2. Model $M_2$.
$$ P_2(ABCD) = P(A)P(B|A)P(C|A)P(D|A) $$

You will notice that the JPD is factorized, and that there are no graphs given. You can reconstruct the graph by reading the families.  

###  Step 1.  Calculate the Conditional Probability tables for both models.
In particular:
* $P(A)$. (used in both models)
* $P(B|A)$. (used in both models)
* $P(C|A)$. (used in both models)
* $P(D|BC)$. (used only in model $M_1$)
* $P(D|A)$. (used only in model $M_2$)

Calculate and display these tables neatly.  I recommend a Pandas dataframe for each table.  It will make step 2 easier!

In [5]:
# Your work here

#### Grading: 5 marks
Your notebook calculates the 5 tables correctly, and presents them neatly.

###  Step 2.  Compare the two models using the likelihood of the data
Let $\mathbf{X}$ represent the dataset.  We want to compare the following two quantities:
* $P(\mathbf{X} | M_1)$ that is, the likelihood of the data using $M_1$.
* $P(\mathbf{X} | M_2)$ that is, the likelihood of the data using the other model.

Let $a,b,c,d$ be the values for the variables on one of the rows.  In the first model:
$$ P(a,b,c,d|M_1) = P(a)P(b|a)P(c|a)P(d|bc) $$
In other words, we are using the tables for $M_1$ in this factorization.  TO finish this calculation off, we have to look up one number from each table, and mutiply them all together.  These are the tables we calculated in Step 1.

This is the likelihood of a single row of the data.  To calculate the likelihood of the whole data set, we assume each row is independent:
$$ P(\mathbf{X}|M_1) = \prod_i P_1(a_i,b_i,c_i,d_i|M_1). $$
In other words we multiply the likelihoods of all the rows together to get the likelihood of the dataset using Model $M_1$.

Keep in mind this calculation is for the first model, $M_1$; a slightly different calculation is needed for the second model $M_2$, because the family for $D$ is different. 

When you have calculated the likelihood for the data in **both models**, we can say that the preferred model is the one where the likelihood is highest.

Here's the task:  Compare the likelihood of the data in both models, and decide which one fits the data better.

In [9]:
# Your work here

#### Grading: 4 marks 
Your calculations are correct.


### Conclusion: 
Which of the two model is the better fit for the data?


(your conclusions here)


#### Grading: 1 mark 
Your conclusion is appropriate.